In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
mort = pd.read_csv('./newdata.csv', header=0) # data about mortgage
mort.head()

,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,pur_prc_amt,DTI Ratio,OUTCOME,Median_state_inc
0,37,75650.0,85,669,N,1707,6000,160000,0.284500,non-default,"40,171"
1,46,390775.0,102,684,N,0,5025,309000,0.000000,non-default,"44,228"
2,30,112500.0,90,662,Y,1812,4800,176450,0.377500,non-default,"49,894"
3,24,85250.0,97,647,N,3395,6934,110000,0.489616,non-default,"43,217"
4,35,114000.0,100,791,N,3801,5504,103000,0.690589,non-default,"57,352"


In [3]:
## First_home 문자형 데이터 변환
mort['First_home'] = np.where(mort['First_home']=='Y',1,0)
mort['First_home']

## OUTCOME 데이터 변환
mort['OUTCOME'] = np.where(mort['OUTCOME']=='non-default',0,1) # non-default = 0, default = 1

## string 타입을 정수로 변환하기
for k in range(0,10607):
    mort.Median_state_inc[k] = np.float64(mort.Median_state_inc[k].replace(",", ""))

outcome = mort.OUTCOME
mort.drop(['OUTCOME'], axis='columns', inplace=True)

mort

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,pur_prc_amt,DTI Ratio,Median_state_inc
0,37,75650.0,85,669,0,1707,6000,160000,0.284500,40171
1,46,390775.0,102,684,0,0,5025,309000,0.000000,44228
2,30,112500.0,90,662,1,1812,4800,176450,0.377500,49894
3,24,85250.0,97,647,0,3395,6934,110000,0.489616,43217
4,35,114000.0,100,791,0,3801,5504,103000,0.690589,57352
...,...,...,...,...,...,...,...,...,...,...
10602,37,280200.0,95,652,1,727,1827,80000,0.397920,42590
10603,30,57000.0,94,589,0,1703,3941,108000,0.432124,45787
10604,33,153200.0,95,675,1,1726,3707,154000,0.465606,43217
10605,38,158850.0,100,786,1,1364,2500,92000,0.545600,40171


In [4]:
# X, y = mort, outcome
## training data와 test data 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mort, outcome, test_size=0.2, random_state=0)

In [6]:
## 순서 중요 (train, test 데이터 나누기 -> train set만 fit_transform)
## Standardization
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

## GridSearchCV & Cross Validation
from sklearn.model_selection import KFold, GridSearchCV

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, f1_score
lr = LogisticRegression(random_state=0)

params = {'penalty':['l1','l2'], 'C':[0.01, 1, 100]}
scoring = {'f1 macro': make_scorer(f1_score , average='macro'), 'f1 micro': make_scorer(f1_score, average = 'micro'), 'Accuracy': make_scorer(accuracy_score) }

gslr = GridSearchCV(lr, params, cv=5, scoring=scoring, refit=False)

In [39]:
X = X_train_std
y = y_train
# train
gslr.fit(X, y)

c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be

c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 1, 100], 'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring={'Accuracy': make_scorer(accuracy_score),
                      'f1 macro': make_scorer(f1_score, average=macro),
                      'f1 micro': make_scorer(f1_score, averag

In [40]:
LRcv_result = pd.DataFrame(gslr.cv_results_)
LRcv_result.to_csv("cv_result_lr.csv")

In [35]:
gslr.score(X_test, y_test)
# pred = gslr.predict(X_test)

NotFittedError: This GridSearchCV instance was initialized with refit=False. score is available only after refitting on the best parameters. You can refit an estimator manually using the ``best_params_`` attribute